In [5]:
from bs4 import BeautifulSoup # библиотека для парсинга - поиск информации по структуре тегов
import requests # получение данных по ссылке
import pandas as pd

html = requests.get('https://attackontitan.fandom.com/ru/wiki/%D0%9A%D0%B0%D1%82%D0%B5%D0%B3%D0%BE%D1%80%D0%B8%D1%8F:%D0%9F%D0%B5%D1%80%D1%81%D0%BE%D0%BD%D0%B0%D0%B6%D0%B8').text
soup = BeautifulSoup(html, 'html')
soup

<!DOCTYPE html>
<html class="client-nojs sse-uxfcp-6047-control" dir="ltr" lang="ru">
<head>
<meta charset="utf-8"/>
<title>Категория:Персонажи | Shingekinokyojin вики | Fandom</title>
<script>document.documentElement.className="client-js sse-uxfcp-6047-control";RLCONF={"wgBreakFrames":false,"wgSeparatorTransformTable":[",\t."," \t,"],"wgDigitTransformTable":["",""],"wgDefaultDateFormat":"dmy","wgMonthNames":["","январь","февраль","март","апрель","май","июнь","июль","август","сентябрь","октябрь","ноябрь","декабрь"],"wgRequestId":"29b37a2de744c7b454cdccd82734cee3","wgCanonicalNamespace":"Category","wgCanonicalSpecialPageName":false,"wgNamespaceNumber":14,"wgPageName":"Категория:Персонажи","wgTitle":"Персонажи","wgCurRevisionId":44665,"wgRevisionId":44665,"wgArticleId":550,"wgIsArticle":true,"wgIsRedirect":false,"wgAction":"view","wgUserName":null,"wgUserGroups":["*"],"wgCategories":["Атака Титанов","Списки Персонажей"],"wgPageViewLanguage":"ru","wgPageContentLanguage":"ru","wgPageConten

In [6]:
soup.find('div', {'class':'NXAUb _T'})

In [7]:
soup.select('[class*="NXAUb"]')

[]

In [8]:
soup.find_all('div', {'class':'NXAUb _T'})

[]

In [9]:
import tqdm
import time

for i in tqdm.tqdm_notebook([1,2,3,5,4], total = 5):
  time.sleep(0.5)

<ipython-input-9-47637a402db6>:4: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for i in tqdm.tqdm_notebook([1,2,3,5,4], total = 5):


  0%|          | 0/5 [00:00<?, ?it/s]

##Найдем ссылки на персонажей

In [10]:
characters = []

for item in soup.find_all('a', {'class': 'category-page__member-link'}):
    name = item.text.strip()
    link = 'https://attackontitan.fandom.com' + item['href']
    characters.append({'name': name, 'link': link})

df = pd.DataFrame(characters)
print(df)

df.to_csv('attackontitan_characters_link.csv', index=False, encoding='utf-8')

                  name                                               link
0        11-й Командор  https://attackontitan.fandom.com/ru/wiki/11-%D...
1                Абель  https://attackontitan.fandom.com/ru/wiki/%D0%9...
2          Абель Рейсс  https://attackontitan.fandom.com/ru/wiki/%D0%9...
3               Азиаты  https://attackontitan.fandom.com/ru/wiki/%D0%9...
4                Альма  https://attackontitan.fandom.com/ru/wiki/%D0%9...
..                 ...                                                ...
195  Фриц (Подставной)  https://attackontitan.fandom.com/ru/wiki/%D0%A...
196          Фэй Йегер  https://attackontitan.fandom.com/ru/wiki/%D0%A...
197         Ханджи Зое  https://attackontitan.fandom.com/ru/wiki/%D0%A...
198      Ханна Диамант  https://attackontitan.fandom.com/ru/wiki/%D0%A...
199             Ханнес  https://attackontitan.fandom.com/ru/wiki/%D0%A...

[200 rows x 2 columns]


## Теперь создадим датасет

Вывелим одного персонажа для удобства поиска

In [ ]:
html = requests.get('https://attackontitan.fandom.com/ru/wiki/%D0%9B%D0%B5%D0%B2%D0%B8_%D0%90%D0%BA%D0%BA%D0%B5%D1%80%D0%BC%D0%B0%D0%BD').text
levi = BeautifulSoup(html, 'html')
levi

<!DOCTYPE html>
<html class="client-nojs sse-other" dir="ltr" lang="ru">
<head>
<meta charset="utf-8"/>
<title>Леви Аккерман | Shingekinokyojin вики | Fandom</title>
<script>document.documentElement.className="client-js sse-other";RLCONF={"wgBreakFrames":false,"wgSeparatorTransformTable":[",\t."," \t,"],"wgDigitTransformTable":["",""],"wgDefaultDateFormat":"dmy","wgMonthNames":["","январь","февраль","март","апрель","май","июнь","июль","август","сентябрь","октябрь","ноябрь","декабрь"],"wgRequestId":"bffa3c9acafb65e4163d3210656d69a6","wgCanonicalNamespace":"","wgCanonicalSpecialPageName":false,"wgNamespaceNumber":0,"wgPageName":"Леви_Аккерман","wgTitle":"Леви Аккерман","wgCurRevisionId":46156,"wgRevisionId":46156,"wgArticleId":505,"wgIsArticle":true,"wgIsRedirect":false,"wgAction":"view","wgUserName":null,"wgUserGroups":["*"],"wgCategories":["Персонажи","Мужчины","Живые"],"wgPageViewLanguage":"ru","wgPageContentLanguage":"ru","wgPageContentModel":"wikitext","wgRelevantPageName":"Леви_Акк

In [11]:
characters = pd.read_csv('attackontitan_characters_link.csv')
items = []

for index, row in characters.iterrows():
    char_name = row['name']
    char_url = row['link']

    response = requests.get(char_url)
    soup = BeautifulSoup(response.text, 'html.parser')

    data = {'name': char_name, 'link': char_url}

    info_sections = soup.find_all('section', {'class': 'pi-item pi-group pi-border-color'})

    for section in info_sections:
        for item in section.find_all('div', {'class': 'pi-item pi-data pi-item-spacing pi-border-color'}):
            key_elem = item.find('h3', {'class': 'pi-data-label pi-secondary-font'})
            value_elem = item.find('div', {'class': 'pi-data-value pi-font'})

            if key_elem and value_elem:
                key = key_elem.text.strip()

                value_links = value_elem.find_all('a')
                if value_links:
                    value = ", ".join(link.text.strip() for link in value_links)
                else:
                    value = value_elem.text.strip()

                data[key] = value

    table = soup.find('table', {'class': 'pi-horizontal-group'})
    if table:
        rows = table.find_all('td', {'class': 'pi-horizontal-group-item pi-data-value pi-font pi-border-color pi-item-spacing'})
        if len(rows) >= 2:
            data['Рост'] = rows[0].text.strip()
            data['Вес'] = rows[1].text.strip()

    titan_kills = soup.find('div', {'data-source': 'убийства'})
    if titan_kills:
        value_elem = titan_kills.find('div', {'class': 'pi-data-value pi-font'})
        if value_elem:
            data['Кол-во убитых титанов'] = value_elem.text.strip()

    rating = soup.find('td', {'data-source': 'оценка'})
    if rating:
        data['Оценка'] = rating.text.strip()

    print(f"Найдено для {char_name}: {data}")

    items.append(data)

    time.sleep(1)

df = pd.DataFrame(items)
df.to_csv('attackontitan_info.csv', index=False, encoding='utf-8')
print(df.head())
pd.DataFrame(items)

Найдено для 11-й Командор: {'name': '11-й Командор', 'link': 'https://attackontitan.fandom.com/ru/wiki/11-%D0%B9_%D0%9A%D0%BE%D0%BC%D0%B0%D0%BD%D0%B4%D0%BE%D1%80'}
Найдено для Абель: {'name': 'Абель', 'link': 'https://attackontitan.fandom.com/ru/wiki/%D0%90%D0%B1%D0%B5%D0%BB%D1%8C', 'Рост': '175 см[2]', 'Вес': '62 кг[2]'}
Найдено для Абель Рейсс: {'name': 'Абель Рейсс', 'link': 'https://attackontitan.fandom.com/ru/wiki/%D0%90%D0%B1%D0%B5%D0%BB%D1%8C_%D0%A0%D0%B5%D0%B9%D1%81%D1%81', 'Пол': 'Женский', 'День рождения': '7 августа', 'Вид': 'Человек', 'Возраст': '12 лет (845)', 'Родственники': 'Семья Фриц, Имир Фриц, Семья Рейсс, Карл Фриц, Г-н Рейсс, Род Рейсс, Леди Рейсс, Фрида Рейсс, Улькин Рейсс, Флориана Рейсс, Дирк Рейсс, Ури Рейсс, Хистория Рейсс', 'Статус': 'Мертва', 'Манга': 'Глава 62 "Грех"', 'Аниме': 'эпизод 42'}
Найдено для Азиаты: {'name': 'Азиаты', 'link': 'https://attackontitan.fandom.com/ru/wiki/%D0%90%D0%B7%D0%B8%D0%B0%D1%82%D1%8B', 'Место нахождения': 'Хизуру', 'Известные 

,name,link,Рост,Вес,Пол,День рождения,Вид,Возраст,Родственники,Статус,...,Каноничная пара,Титан,Организация до,Количество убийств,Место,Назначение,Тип,Пользователи,Известная информация,Характерные СпособностиТитана
0,11-й Командор,https://attackontitan.fandom.com/ru/wiki/11-%D...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Абель,https://attackontitan.fandom.com/ru/wiki/%D0%9...,175 см[2],62 кг[2],NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Абель Рейсс,https://attackontitan.fandom.com/ru/wiki/%D0%9...,NaN,NaN,Женский,7 августа,Человек,12 лет (845),"Семья Фриц, Имир Фриц, Семья Рейсс, Карл Фриц,...",Мертва,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Азиаты,https://attackontitan.fandom.com/ru/wiki/%D0%9...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Альма,https://attackontitan.fandom.com/ru/wiki/%D0%9...,168 см,54 кг,Женский,14 июля,Человек,NaN,"Хистория Рейсс, Род Рейсс",Мертва,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,Фриц (Подставной),https://attackontitan.fandom.com/ru/wiki/%D0%A...,177 см,62 кг,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
196,Фэй Йегер,https://attackontitan.fandom.com/ru/wiki/%D0%A...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
197,Ханджи Зое,https://attackontitan.fandom.com/ru/wiki/%D0%A...,170 см [1],60 кг [1],NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
198,Ханна Диамант,https://attackontitan.fandom.com/ru/wiki/%D0%A...,154 см,50 кг,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## Теперь смотрим строки, столбцы и убираем ненужные (чистим датасет)

In [12]:
df.columns

Index(['name', 'link', 'Рост', 'Вес', 'Пол', 'День рождения', 'Вид', 'Возраст',
       'Родственники', 'Статус', 'Манга', 'Аниме', 'Место нахождения',
       'Известные семьи', 'Происхождение', 'Кол-во убитых титанов', 'Оценка',
       'Организация', 'Профессия', 'Имя сейю', 'Звание', 'Каноничная пара',
       'Титан', 'Организация до', 'Количество убийств', 'Место', 'Назначение',
       'Тип', 'Пользователи', 'Известная информация',
       'Характерные СпособностиТитана'],
      dtype='object')

In [13]:
df = df.drop(['День рождения', 'Родственники', 'Манга', 'Аниме', 'Место нахождения','Известные семьи', 'Вид', 'Происхождение',
              'Оценка', 'Организация', 'Профессия', 'Имя сейю', 'Звание', 'Каноничная пара','Титан', 'Организация до', 'Возраст',
              'Количество убийств', 'Место', 'Назначение', 'Тип', 'Пользователи', 'Известная информация', 'Характерные СпособностиТитана'], axis=1)
df

,name,link,Рост,Вес,Пол,Статус,Кол-во убитых титанов
0,11-й Командор,https://attackontitan.fandom.com/ru/wiki/11-%D...,NaN,NaN,NaN,NaN,NaN
1,Абель,https://attackontitan.fandom.com/ru/wiki/%D0%9...,175 см[2],62 кг[2],NaN,NaN,NaN
2,Абель Рейсс,https://attackontitan.fandom.com/ru/wiki/%D0%9...,NaN,NaN,Женский,Мертва,NaN
3,Азиаты,https://attackontitan.fandom.com/ru/wiki/%D0%9...,NaN,NaN,NaN,NaN,NaN
4,Альма,https://attackontitan.fandom.com/ru/wiki/%D0%9...,168 см,54 кг,Женский,Мертва,NaN
...,...,...,...,...,...,...,...
195,Фриц (Подставной),https://attackontitan.fandom.com/ru/wiki/%D0%A...,177 см,62 кг,NaN,NaN,NaN
196,Фэй Йегер,https://attackontitan.fandom.com/ru/wiki/%D0%A...,NaN,NaN,NaN,NaN,NaN
197,Ханджи Зое,https://attackontitan.fandom.com/ru/wiki/%D0%A...,170 см [1],60 кг [1],NaN,NaN,Соло: Минимум 5 [2]В команде: НеизвестноВсего:...
198,Ханна Диамант,https://attackontitan.fandom.com/ru/wiki/%D0%A...,154 см,50 кг,NaN,NaN,NaN


In [14]:
# Удаляем там, где все пусто кроме имени и ссылки
df = df.dropna(how='all', subset=[col for col in df.columns if col not in ['name', 'link']])
df

,name,link,Рост,Вес,Пол,Статус,Кол-во убитых титанов
1,Абель,https://attackontitan.fandom.com/ru/wiki/%D0%9...,175 см[2],62 кг[2],NaN,NaN,NaN
2,Абель Рейсс,https://attackontitan.fandom.com/ru/wiki/%D0%9...,NaN,NaN,Женский,Мертва,NaN
4,Альма,https://attackontitan.fandom.com/ru/wiki/%D0%9...,168 см,54 кг,Женский,Мертва,NaN
5,Анка Рейнбергер,https://attackontitan.fandom.com/ru/wiki/%D0%9...,155 см[1],53 кг[1],NaN,NaN,NaN
7,Армин Арлерт,https://attackontitan.fandom.com/ru/wiki/%D0%9...,• 163 см [850] [1]• 169 см [854] [4]• 60 м [Ти...,• 55 кг [850] [1]• 56 кг [854],NaN,NaN,Соло: Минимум 2 [8]В команде: Неизвестно Всего...
...,...,...,...,...,...,...,...
193,Фрида Рейсс,https://attackontitan.fandom.com/ru/wiki/%D0%A...,171 см (Человек)[6]13 м (Титан)[7],57 кг[6],NaN,NaN,NaN
195,Фриц (Подставной),https://attackontitan.fandom.com/ru/wiki/%D0%A...,177 см,62 кг,NaN,NaN,NaN
197,Ханджи Зое,https://attackontitan.fandom.com/ru/wiki/%D0%A...,170 см [1],60 кг [1],NaN,NaN,Соло: Минимум 5 [2]В команде: НеизвестноВсего:...
198,Ханна Диамант,https://attackontitan.fandom.com/ru/wiki/%D0%A...,154 см,50 кг,NaN,NaN,NaN


In [15]:
# Сохраним в датасет
df.to_csv('clean_info.csv', index=False, encoding='utf-8')

## Дополним потерянные значения

In [16]:
# Для начала посмотрим, сколько значений мы потеряли
height_count = (df["Рост"].isnull()).sum()
print(f"Количество NULL в 'Рост': {height_count}")

weight_count = (df["Вес"].isnull()).sum()
print(f"Количество NULL в 'Вес': {weight_count}")

gender_count = (df["Пол"].isnull()).sum()
print(f"Количество NULL в 'Пол': {gender_count}")

status_count = (df["Статус"].isnull()).sum()
print(f"Количество NULL в 'Статус': {status_count}")

titan_count = (df["Кол-во убитых титанов"].isnull()).sum()
print(f"Количество NULL в 'Кол-во убитых титанов': {titan_count}")


Количество NULL в 'Рост': 60
Количество NULL в 'Вес': 60
Количество NULL в 'Пол': 75
Количество NULL в 'Статус': 74
Количество NULL в 'Кол-во убитых титанов': 119


### Рост и Вес

In [17]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import re
import time

df = pd.read_csv('clean_info.csv')

# Функция для извлечения роста и веса
def extract_height_weight(soup):
    height = None
    weight = None

    # Находим все строки с данными о росте и весе
    rows = soup.find_all('tr')

    for row in rows:
        height_td = row.find('td', {'data-source': 'рост'})
        weight_td = row.find('td', {'data-source': 'вес'})

        # Извлекаем рост
        if height_td:
            height_text = height_td.text.strip()
            height = parse_height(height_text)

        # Извлекаем вес
        if weight_td:
            weight_text = weight_td.text.strip()
            weight = parse_weight(weight_text)

    return height, weight

# Функция для преобразования текста о росте
def parse_height(height_text):
    # Преобразуем рост в сантиметры
    match = re.search(r'(\d+)\s*(см|м)', height_text)
    if match:
        value = int(match.group(1))
        unit = match.group(2)
        if unit == 'м':
            return value * 100
        return value
    return None

# Функция для преобразования текста о весе
def parse_weight(weight_text):
    # Извлекаем только числа и конвертируем в int
    match = re.search(r'(\d+)\s*кг', weight_text)
    if match:
        return int(match.group(1))
    return None

# Отбираем строки, где "Рост" или "Вес" отсутствуют
missing_data_df = df[df["Рост"].isnull()]

for index, row in missing_data_df.iterrows():
    char_name = row['name']
    char_url = row['link']

    if pd.isna(char_url) or not char_url.startswith("http"):
        print(f"Пропущено: {char_name} (отсутствует корректный URL)")
        continue

    try:
        response = requests.get(char_url)
        response.raise_for_status()
        soup = BeautifulSoup(response.text, 'html.parser')

        # Извлекаем рост и вес
        height, weight = extract_height_weight(soup)

        # Обновляем данные в DataFrame
        if height is not None:
            df.at[index, "Рост"] = height
            print(f"Обновлено: {char_name} -> Рост: {height} см")

        if weight is not None:
            df.at[index, "Вес"] = weight
            print(f"Обновлено: {char_name} -> Вес: {weight} кг")

    except requests.RequestException as e:
        print(f"Ошибка запроса для {char_name}: {e}")

    time.sleep(1)

df.to_csv('height_weight_info.csv', index=False, encoding='utf-8')

# Сортировка по росту и весу
df_sorted = df.sort_values(by=["Рост", "Вес"], ascending=[True, True])
df

,name,link,Рост,Вес,Пол,Статус,Кол-во убитых титанов
0,Абель,https://attackontitan.fandom.com/ru/wiki/%D0%9...,175 см[2],62 кг[2],NaN,NaN,NaN
1,Абель Рейсс,https://attackontitan.fandom.com/ru/wiki/%D0%9...,NaN,NaN,Женский,Мертва,NaN
2,Альма,https://attackontitan.fandom.com/ru/wiki/%D0%9...,168 см,54 кг,Женский,Мертва,NaN
3,Анка Рейнбергер,https://attackontitan.fandom.com/ru/wiki/%D0%9...,155 см[1],53 кг[1],NaN,NaN,NaN
4,Армин Арлерт,https://attackontitan.fandom.com/ru/wiki/%D0%9...,• 163 см [850] [1]• 169 см [854] [4]• 60 м [Ти...,• 55 кг [850] [1]• 56 кг [854],NaN,NaN,Соло: Минимум 2 [8]В команде: Неизвестно Всего...
...,...,...,...,...,...,...,...
139,Фрида Рейсс,https://attackontitan.fandom.com/ru/wiki/%D0%A...,171 см (Человек)[6]13 м (Титан)[7],57 кг[6],NaN,NaN,NaN
140,Фриц (Подставной),https://attackontitan.fandom.com/ru/wiki/%D0%A...,177 см,62 кг,NaN,NaN,NaN
141,Ханджи Зое,https://attackontitan.fandom.com/ru/wiki/%D0%A...,170 см [1],60 кг [1],NaN,NaN,Соло: Минимум 5 [2]В команде: НеизвестноВсего:...
142,Ханна Диамант,https://attackontitan.fandom.com/ru/wiki/%D0%A...,154 см,50 кг,NaN,NaN,NaN


In [18]:
height_count = (df["Рост"].isnull()).sum()
print(f"Количество NULL в 'Рост': {height_count}")

weight_count = (df["Вес"].isnull()).sum()
print(f"Количество NULL в 'Вес': {weight_count}")

Количество NULL в 'Рост': 60
Количество NULL в 'Вес': 60


Мы видим, что ничего не изменилось

###Пол

In [19]:
df = pd.read_csv('clean_info.csv')

# Отбираем строки, где "Пол" отсутствует
missing_gender_df = df[df["Пол"].isnull()]

for index, row in missing_gender_df.iterrows():
  char_name = row['name']
  char_url = row['link']

  if pd.isna(char_url) or not char_url.startswith("http"):
    print(f"Пропущено: {char_name} (отсутствует корректный URL)")
    continue

  try:
    response = requests.get(char_url)
    response.raise_for_status()
    soup = BeautifulSoup(response.text, 'html.parser')

    # Ищем все div-элементы с классом "pi-data-value pi-font"
    all_divs = soup.find_all('div', class_='pi-data-value pi-font')

    print(f"---char_name---")

    for div in all_divs:
      gender_text = div.text.strip()

      if gender_text.lower() in ["мужской", "женский"]:
        df.at[index, "Пол"] = gender_text.capitalize()
        print(f"--- Обновлено: {char_name} -> {gender_text.capitalize()}")
        break

  except requests.RequestException as e:
    print(f"Ошибка запроса для {char_name}: {e}")

  time.sleep(1)

df.to_csv('gender_info_updated.csv', index=False, encoding='utf-8')
df

---char_name---
--- Обновлено: Абель -> Мужской
---char_name---
--- Обновлено: Анка Рейнбергер -> Женский
---char_name---
--- Обновлено: Армин Арлерт -> Мужской
---char_name---
--- Обновлено: Артур Браус -> Мужской
---char_name---
--- Обновлено: Бертольд Гувер -> Мужской
---char_name---
--- Обновлено: Вилли Тайбер -> Мужской
---char_name---
--- Обновлено: Г-н Рал -> Мужской
---char_name---
--- Обновлено: Г-н Смит -> Мужской
---char_name---
--- Обновлено: Габи Браун -> Женский
---char_name---
--- Обновлено: Гергер -> Мужской
---char_name---
--- Обновлено: Гриша Йегер -> Мужской
---char_name---
--- Обновлено: Гюнтер Шульц -> Мужской
---char_name---
--- Обновлено: Даз -> Мужской
---char_name---
--- Обновлено: Дариус Закклай -> Мужской
---char_name---
--- Обновлено: Дед Аккерман -> Мужской
---char_name---
---char_name---
--- Обновлено: Джер Санес -> Мужской
---char_name---
--- Обновлено: Дина Фриц -> Женский
---char_name---
--- Обновлено: Дирк -> Мужской
---char_name---
--- Обновлено: Дот 

,name,link,Рост,Вес,Пол,Статус,Кол-во убитых титанов
0,Абель,https://attackontitan.fandom.com/ru/wiki/%D0%9...,175 см[2],62 кг[2],Мужской,NaN,NaN
1,Абель Рейсс,https://attackontitan.fandom.com/ru/wiki/%D0%9...,NaN,NaN,Женский,Мертва,NaN
2,Альма,https://attackontitan.fandom.com/ru/wiki/%D0%9...,168 см,54 кг,Женский,Мертва,NaN
3,Анка Рейнбергер,https://attackontitan.fandom.com/ru/wiki/%D0%9...,155 см[1],53 кг[1],Женский,NaN,NaN
4,Армин Арлерт,https://attackontitan.fandom.com/ru/wiki/%D0%9...,• 163 см [850] [1]• 169 см [854] [4]• 60 м [Ти...,• 55 кг [850] [1]• 56 кг [854],Мужской,NaN,Соло: Минимум 2 [8]В команде: Неизвестно Всего...
...,...,...,...,...,...,...,...
139,Фрида Рейсс,https://attackontitan.fandom.com/ru/wiki/%D0%A...,171 см (Человек)[6]13 м (Титан)[7],57 кг[6],Женский,NaN,NaN
140,Фриц (Подставной),https://attackontitan.fandom.com/ru/wiki/%D0%A...,177 см,62 кг,Мужской,NaN,NaN
141,Ханджи Зое,https://attackontitan.fandom.com/ru/wiki/%D0%A...,170 см [1],60 кг [1],NaN,NaN,Соло: Минимум 5 [2]В команде: НеизвестноВсего:...
142,Ханна Диамант,https://attackontitan.fandom.com/ru/wiki/%D0%A...,154 см,50 кг,Женский,NaN,NaN


In [21]:
gender_count = (df["Пол"].isnull()).sum()
print(f"Количество NULL в 'Пол': {gender_count}")

Количество NULL в 'Пол': 4


### Статус

In [38]:
df = pd.read_csv('gender_info_updated.csv')

# Отбираем строки, где "Статус" отсутствует
missing_status_df = df[df["Статус"].isnull()]

for index, row in missing_status_df.iterrows():
  char_name = row['name']
  char_url = row['link']

  if pd.isna(char_url) or not char_url.startswith("http"):
    print(f"Пропущено: {char_name} (отсутствует корректный URL)")
    continue

  try:
    response = requests.get(char_url)
    response.raise_for_status()
    soup = BeautifulSoup(response.text, 'html.parser')

    # Ищем все div-элементы с классом "pi-data-value pi-font"
    all_divs = soup.find_all('div', class_='pi-data-value pi-font')

    print(f"---char_name---")

    for div in all_divs:
      status_text = div.text.strip()

      if status_text.lower() in ["мертв", "жив", "мертва", "жива"]:
        df.at[index, "Статус"] = status_text.capitalize()
        print(f"--- Обновлено: {char_name} -> {status_text.capitalize()}")
        break

  except requests.RequestException as e:
    print(f"Ошибка запроса для {char_name}: {e}")

  time.sleep(1)

df.to_csv('status_info_updated.csv', index=False, encoding='utf-8')
df

FileNotFoundError: [Errno 2] No such file or directory: 'gender_info_updated.csv'

In [23]:
status_count = (df["Статус"].isnull()).sum()
print(f"Количество NULL в 'Статус': {status_count}")

Количество NULL в 'Статус': 51


###Кол-во убитых титанов

Здесь мы прекрасно понимаем, что не все убивали титанов, в основном развед корпус, поэтому не ожидаем больших изменений или их вообще не будет

In [24]:
# Функция для извлечения количества убитых титанов
def extract_killed_titans(soup):
    # Ищем div с классом 'pi-data-value pi-font' и внутри которого есть информация о количестве убитых титанов
    killed_titans_div = soup.find('div', class_='pi-data-value pi-font')

    if killed_titans_div:
        text = killed_titans_div.get_text(strip=True)

        match = re.search(r'Минимум\s*(\d+)', text)
        if match:
            return int(match.group(1))

        if "Неизвестно" in text:
            return None

    return None

df = pd.read_csv('clean_info.csv')
missing_kills_df = df[df["Кол-во убитых титанов"].isnull()]

for index, row in missing_kills_df.iterrows():
    char_name = row['name']
    char_url = row['link']

    if pd.isna(char_url) or not char_url.startswith("http"):
        print(f"Пропущено: {char_name} (отсутствует корректный URL)")
        continue

    try:
        response = requests.get(char_url)
        response.raise_for_status()
        soup = BeautifulSoup(response.text, 'html.parser')

        # Извлекаем количество убитых титанов
        killed_titans = extract_killed_titans(soup)

        # Обновляем данные в DataFrame
        if killed_titans is not None:
            df.at[index, "Кол-во убитых титанов"] = killed_titans
            print(f"Обновлено: {char_name} -> Кол-во убитых титанов: {killed_titans}")
        else:
            print(f"Неизвестно для {char_name}")

    except requests.RequestException as e:
        print(f"Ошибка запроса для {char_name}: {e}")

    time.sleep(1)

df.to_csv('_info_with_kills.csv', index=False, encoding='utf-8')
df

Неизвестно для Абель
Неизвестно для Абель Рейсс
Неизвестно для Альма
Неизвестно для Анка Рейнбергер
Неизвестно для Артур Браус
Неизвестно для Аурилл
Неизвестно для Безымянный медик
Неизвестно для Валдо Рихтер
Неизвестно для Вальц
Неизвестно для Вилли Тайбер
Неизвестно для Вэйн Эйснер
Неизвестно для Г-жа Аккерман
Неизвестно для Г-жа Арлерт
Неизвестно для Г-жа Браун
Неизвестно для Г-жа Кирштейн
Неизвестно для Г-жа Спрингер
Неизвестно для Г-н Аккерман
Неизвестно для Г-н Арлерт
Неизвестно для Г-н Рал
Неизвестно для Г-н Рейсс
Неизвестно для Г-н Смит
Неизвестно для Г-н Спрингер
Неизвестно для Герос
Неизвестно для Гордон
Неизвестно для Гриша Йегер
Неизвестно для Гросс
Неизвестно для Густав
Неизвестно для Даз
Неизвестно для Дариус Байер-Варбрун
Неизвестно для Дариус Закклай
Неизвестно для Девушка из Культа стен
Неизвестно для Дед Аккерман
Неизвестно для Дед Арлерт
Неизвестно для Делтофф
Неизвестно для Демонический титан
Неизвестно для Деннис Айбрингер
Неизвестно для Джер Санес
Неизвестно для Д

,name,link,Рост,Вес,Пол,Статус,Кол-во убитых титанов
0,Абель,https://attackontitan.fandom.com/ru/wiki/%D0%9...,175 см[2],62 кг[2],NaN,NaN,NaN
1,Абель Рейсс,https://attackontitan.fandom.com/ru/wiki/%D0%9...,NaN,NaN,Женский,Мертва,NaN
2,Альма,https://attackontitan.fandom.com/ru/wiki/%D0%9...,168 см,54 кг,Женский,Мертва,NaN
3,Анка Рейнбергер,https://attackontitan.fandom.com/ru/wiki/%D0%9...,155 см[1],53 кг[1],NaN,NaN,NaN
4,Армин Арлерт,https://attackontitan.fandom.com/ru/wiki/%D0%9...,• 163 см [850] [1]• 169 см [854] [4]• 60 м [Ти...,• 55 кг [850] [1]• 56 кг [854],NaN,NaN,Соло: Минимум 2 [8]В команде: Неизвестно Всего...
...,...,...,...,...,...,...,...
139,Фрида Рейсс,https://attackontitan.fandom.com/ru/wiki/%D0%A...,171 см (Человек)[6]13 м (Титан)[7],57 кг[6],NaN,NaN,NaN
140,Фриц (Подставной),https://attackontitan.fandom.com/ru/wiki/%D0%A...,177 см,62 кг,NaN,NaN,NaN
141,Ханджи Зое,https://attackontitan.fandom.com/ru/wiki/%D0%A...,170 см [1],60 кг [1],NaN,NaN,Соло: Минимум 5 [2]В команде: НеизвестноВсего:...
142,Ханна Диамант,https://attackontitan.fandom.com/ru/wiki/%D0%A...,154 см,50 кг,NaN,NaN,NaN


In [25]:
titan_count = (df["Кол-во убитых титанов"].isnull()).sum()
print(f"Количество NULL в 'Кол-во убитых титанов': {titan_count}")

Количество NULL в 'Кол-во убитых титанов': 119


In [26]:
# Проверка
height_count = (df["Рост"].isnull()).sum()
print(f"Количество NULL в 'Рост': {height_count}")

weight_count = (df["Вес"].isnull()).sum()
print(f"Количество NULL в 'Вес': {weight_count}")

gender_count = (df["Пол"].isnull()).sum()
print(f"Количество NULL в 'Пол': {gender_count}")

status_count = (df["Статус"].isnull()).sum()
print(f"Количество NULL в 'Статус': {status_count}")

titan_count = (df["Кол-во убитых титанов"].isnull()).sum()
print(f"Количество NULL в 'Кол-во убитых титанов': {titan_count}")


Количество NULL в 'Рост': 60
Количество NULL в 'Вес': 60
Количество NULL в 'Пол': 75
Количество NULL в 'Статус': 74
Количество NULL в 'Кол-во убитых титанов': 119


Как и ожидалось ничего не изменилось

##Теперь займемся значениями в ячейках

In [27]:
df.to_csv('parse_info.csv', index=False, encoding='utf-8')
df

,name,link,Рост,Вес,Пол,Статус,Кол-во убитых титанов
0,Абель,https://attackontitan.fandom.com/ru/wiki/%D0%9...,175 см[2],62 кг[2],NaN,NaN,NaN
1,Абель Рейсс,https://attackontitan.fandom.com/ru/wiki/%D0%9...,NaN,NaN,Женский,Мертва,NaN
2,Альма,https://attackontitan.fandom.com/ru/wiki/%D0%9...,168 см,54 кг,Женский,Мертва,NaN
3,Анка Рейнбергер,https://attackontitan.fandom.com/ru/wiki/%D0%9...,155 см[1],53 кг[1],NaN,NaN,NaN
4,Армин Арлерт,https://attackontitan.fandom.com/ru/wiki/%D0%9...,• 163 см [850] [1]• 169 см [854] [4]• 60 м [Ти...,• 55 кг [850] [1]• 56 кг [854],NaN,NaN,Соло: Минимум 2 [8]В команде: Неизвестно Всего...
...,...,...,...,...,...,...,...
139,Фрида Рейсс,https://attackontitan.fandom.com/ru/wiki/%D0%A...,171 см (Человек)[6]13 м (Титан)[7],57 кг[6],NaN,NaN,NaN
140,Фриц (Подставной),https://attackontitan.fandom.com/ru/wiki/%D0%A...,177 см,62 кг,NaN,NaN,NaN
141,Ханджи Зое,https://attackontitan.fandom.com/ru/wiki/%D0%A...,170 см [1],60 кг [1],NaN,NaN,Соло: Минимум 5 [2]В команде: НеизвестноВсего:...
142,Ханна Диамант,https://attackontitan.fandom.com/ru/wiki/%D0%A...,154 см,50 кг,NaN,NaN,NaN


### Рост

In [28]:
import pandas as pd
import re

def clean_height(height):
  # Если NaN, просто возвращаем его
  if pd.isna(height):
      return height

  # Извлекаем все числовые значения с "см" перед ссылками или пояснениями
  matches = re.findall(r'(\d+)\s*см', height)

  # Если есть найденные значения, берём последнее найденное значение и преобразуем в int
  if matches:
      return int(matches[-1])

  return None  # Если ничего не найдено

df["Рост"] = df["Рост"].apply(clean_height).astype("Int64")
df.to_csv('height_fields_info.csv', index=False, encoding='utf-8')
df

,name,link,Рост,Вес,Пол,Статус,Кол-во убитых титанов
0,Абель,https://attackontitan.fandom.com/ru/wiki/%D0%9...,175,62 кг[2],NaN,NaN,NaN
1,Абель Рейсс,https://attackontitan.fandom.com/ru/wiki/%D0%9...,<NA>,NaN,Женский,Мертва,NaN
2,Альма,https://attackontitan.fandom.com/ru/wiki/%D0%9...,168,54 кг,Женский,Мертва,NaN
3,Анка Рейнбергер,https://attackontitan.fandom.com/ru/wiki/%D0%9...,155,53 кг[1],NaN,NaN,NaN
4,Армин Арлерт,https://attackontitan.fandom.com/ru/wiki/%D0%9...,169,• 55 кг [850] [1]• 56 кг [854],NaN,NaN,Соло: Минимум 2 [8]В команде: Неизвестно Всего...
...,...,...,...,...,...,...,...
139,Фрида Рейсс,https://attackontitan.fandom.com/ru/wiki/%D0%A...,171,57 кг[6],NaN,NaN,NaN
140,Фриц (Подставной),https://attackontitan.fandom.com/ru/wiki/%D0%A...,177,62 кг,NaN,NaN,NaN
141,Ханджи Зое,https://attackontitan.fandom.com/ru/wiki/%D0%A...,170,60 кг [1],NaN,NaN,Соло: Минимум 5 [2]В команде: НеизвестноВсего:...
142,Ханна Диамант,https://attackontitan.fandom.com/ru/wiki/%D0%A...,154,50 кг,NaN,NaN,NaN


In [29]:
df.to_csv('height_fields_info.csv', index=False, encoding='utf-8')
df

,name,link,Рост,Вес,Пол,Статус,Кол-во убитых титанов
0,Абель,https://attackontitan.fandom.com/ru/wiki/%D0%9...,175,62 кг[2],NaN,NaN,NaN
1,Абель Рейсс,https://attackontitan.fandom.com/ru/wiki/%D0%9...,<NA>,NaN,Женский,Мертва,NaN
2,Альма,https://attackontitan.fandom.com/ru/wiki/%D0%9...,168,54 кг,Женский,Мертва,NaN
3,Анка Рейнбергер,https://attackontitan.fandom.com/ru/wiki/%D0%9...,155,53 кг[1],NaN,NaN,NaN
4,Армин Арлерт,https://attackontitan.fandom.com/ru/wiki/%D0%9...,169,• 55 кг [850] [1]• 56 кг [854],NaN,NaN,Соло: Минимум 2 [8]В команде: Неизвестно Всего...
...,...,...,...,...,...,...,...
139,Фрида Рейсс,https://attackontitan.fandom.com/ru/wiki/%D0%A...,171,57 кг[6],NaN,NaN,NaN
140,Фриц (Подставной),https://attackontitan.fandom.com/ru/wiki/%D0%A...,177,62 кг,NaN,NaN,NaN
141,Ханджи Зое,https://attackontitan.fandom.com/ru/wiki/%D0%A...,170,60 кг [1],NaN,NaN,Соло: Минимум 5 [2]В команде: НеизвестноВсего:...
142,Ханна Диамант,https://attackontitan.fandom.com/ru/wiki/%D0%A...,154,50 кг,NaN,NaN,NaN


###Вес

In [30]:
import pandas as pd
import re

def clean_weight(weight):
  # Если NaN, просто возвращаем его
  if pd.isna(weight):
      return weight

  # Извлекаем все числовые значения с "см" перед ссылками или пояснениями
  matches = re.findall(r'(\d+)\s*кг', weight)

  # Если есть найденные значения, берём последнее найденное значение и преобразуем в int
  if matches:
      return int(matches[-1])

  return None  # Если ничего не найдено

df["Вес"] = df["Вес"].apply(clean_weight).astype("Int64")
df.to_csv('weight_fields_info.csv', index=False, encoding='utf-8')
df

,name,link,Рост,Вес,Пол,Статус,Кол-во убитых титанов
0,Абель,https://attackontitan.fandom.com/ru/wiki/%D0%9...,175,62,NaN,NaN,NaN
1,Абель Рейсс,https://attackontitan.fandom.com/ru/wiki/%D0%9...,<NA>,<NA>,Женский,Мертва,NaN
2,Альма,https://attackontitan.fandom.com/ru/wiki/%D0%9...,168,54,Женский,Мертва,NaN
3,Анка Рейнбергер,https://attackontitan.fandom.com/ru/wiki/%D0%9...,155,53,NaN,NaN,NaN
4,Армин Арлерт,https://attackontitan.fandom.com/ru/wiki/%D0%9...,169,56,NaN,NaN,Соло: Минимум 2 [8]В команде: Неизвестно Всего...
...,...,...,...,...,...,...,...
139,Фрида Рейсс,https://attackontitan.fandom.com/ru/wiki/%D0%A...,171,57,NaN,NaN,NaN
140,Фриц (Подставной),https://attackontitan.fandom.com/ru/wiki/%D0%A...,177,62,NaN,NaN,NaN
141,Ханджи Зое,https://attackontitan.fandom.com/ru/wiki/%D0%A...,170,60,NaN,NaN,Соло: Минимум 5 [2]В команде: НеизвестноВсего:...
142,Ханна Диамант,https://attackontitan.fandom.com/ru/wiki/%D0%A...,154,50,NaN,NaN,NaN


### Статус

"Пол" в порядке. Но можно привести к значениям статус:

Мертв, мертва -> Мертв

Жив, Жива -> Жив

In [31]:
import pandas as pd

def normalize_status(status):
    if pd.isna(status):
        return status  # Оставляем NaN без изменений
    status = status.lower().strip()

    if status in ["мертв", "мертва"]:
        return "Мертв"
    elif status in ["жив", "жива"]:
        return "Жив"
    return status

df["Статус"] = df["Статус"].apply(normalize_status)
df.to_csv('status_info.csv', index=False, encoding='utf-8')
df

,name,link,Рост,Вес,Пол,Статус,Кол-во убитых титанов
0,Абель,https://attackontitan.fandom.com/ru/wiki/%D0%9...,175,62,NaN,NaN,NaN
1,Абель Рейсс,https://attackontitan.fandom.com/ru/wiki/%D0%9...,<NA>,<NA>,Женский,Мертв,NaN
2,Альма,https://attackontitan.fandom.com/ru/wiki/%D0%9...,168,54,Женский,Мертв,NaN
3,Анка Рейнбергер,https://attackontitan.fandom.com/ru/wiki/%D0%9...,155,53,NaN,NaN,NaN
4,Армин Арлерт,https://attackontitan.fandom.com/ru/wiki/%D0%9...,169,56,NaN,NaN,Соло: Минимум 2 [8]В команде: Неизвестно Всего...
...,...,...,...,...,...,...,...
139,Фрида Рейсс,https://attackontitan.fandom.com/ru/wiki/%D0%A...,171,57,NaN,NaN,NaN
140,Фриц (Подставной),https://attackontitan.fandom.com/ru/wiki/%D0%A...,177,62,NaN,NaN,NaN
141,Ханджи Зое,https://attackontitan.fandom.com/ru/wiki/%D0%A...,170,60,NaN,NaN,Соло: Минимум 5 [2]В команде: НеизвестноВсего:...
142,Ханна Диамант,https://attackontitan.fandom.com/ru/wiki/%D0%A...,154,50,NaN,NaN,NaN


### Кол-во убитых титанов

Здесь можно одну колонку разбить на три (Соло, В команде, Всего)

In [32]:
import pandas as pd
import re

def kills(text, category):
    if pd.isna(text):
        return None  # Если NaN, оставляем как есть
    match = re.search(fr'{category}: (?:Минимум )?(\d+)', text)
    return int(match.group(1)) if match else None  # Возвращаем число или None, если не найдено

# Создаём новые колонки
df["Убито титанов Соло"] = df["Кол-во убитых титанов"].apply(lambda x: kills(x, "Соло")).astype("Int64")
df["Убито титанов В команде"] = df["Кол-во убитых титанов"].apply(lambda x: kills(x, "В команде")).astype("Int64")
df["Убито титанов Всего"] = df["Кол-во убитых титанов"].apply(lambda x: kills(x, "Всего")).astype("Int64")

# Удаляем старую колонку
df.drop(columns=["Кол-во убитых титанов"], inplace=True)

# Сохраняем результат
df.to_csv('final_attackontitan_characters_info.csv', index=False, encoding='utf-8')
df

,name,link,Рост,Вес,Пол,Статус,Убито титанов Соло,Убито титанов В команде,Убито титанов Всего
0,Абель,https://attackontitan.fandom.com/ru/wiki/%D0%9...,175,62,NaN,NaN,<NA>,<NA>,<NA>
1,Абель Рейсс,https://attackontitan.fandom.com/ru/wiki/%D0%9...,<NA>,<NA>,Женский,Мертв,<NA>,<NA>,<NA>
2,Альма,https://attackontitan.fandom.com/ru/wiki/%D0%9...,168,54,Женский,Мертв,<NA>,<NA>,<NA>
3,Анка Рейнбергер,https://attackontitan.fandom.com/ru/wiki/%D0%9...,155,53,NaN,NaN,<NA>,<NA>,<NA>
4,Армин Арлерт,https://attackontitan.fandom.com/ru/wiki/%D0%9...,169,56,NaN,NaN,2,<NA>,2
...,...,...,...,...,...,...,...,...,...
139,Фрида Рейсс,https://attackontitan.fandom.com/ru/wiki/%D0%A...,171,57,NaN,NaN,<NA>,<NA>,<NA>
140,Фриц (Подставной),https://attackontitan.fandom.com/ru/wiki/%D0%A...,177,62,NaN,NaN,<NA>,<NA>,<NA>
141,Ханджи Зое,https://attackontitan.fandom.com/ru/wiki/%D0%A...,170,60,NaN,NaN,5,<NA>,5
142,Ханна Диамант,https://attackontitan.fandom.com/ru/wiki/%D0%A...,154,50,NaN,NaN,<NA>,<NA>,<NA>


In [34]:
df.to_csv('attackontitan_characters_info.csv', index=False, encoding='utf-8')

##Добавим данные

In [36]:
import pandas as pd
df = pd.read_csv("attackontitan_characters_info.csv")
df

,name,link,Рост,Вес,Пол,Статус,Убито титанов Соло,Убито титанов В команде,Убито титанов Всего
0,Абель,https://attackontitan.fandom.com/ru/wiki/%D0%9...,175.0,62.0,NaN,NaN,NaN,NaN,NaN
1,Абель Рейсс,https://attackontitan.fandom.com/ru/wiki/%D0%9...,NaN,NaN,Женский,Мертв,NaN,NaN,NaN
2,Альма,https://attackontitan.fandom.com/ru/wiki/%D0%9...,168.0,54.0,Женский,Мертв,NaN,NaN,NaN
3,Анка Рейнбергер,https://attackontitan.fandom.com/ru/wiki/%D0%9...,155.0,53.0,NaN,NaN,NaN,NaN,NaN
4,Армин Арлерт,https://attackontitan.fandom.com/ru/wiki/%D0%9...,169.0,56.0,NaN,NaN,2.0,NaN,2.0
...,...,...,...,...,...,...,...,...,...
139,Фрида Рейсс,https://attackontitan.fandom.com/ru/wiki/%D0%A...,171.0,57.0,NaN,NaN,NaN,NaN,NaN
140,Фриц (Подставной),https://attackontitan.fandom.com/ru/wiki/%D0%A...,177.0,62.0,NaN,NaN,NaN,NaN,NaN
141,Ханджи Зое,https://attackontitan.fandom.com/ru/wiki/%D0%A...,170.0,60.0,NaN,NaN,5.0,NaN,5.0
142,Ханна Диамант,https://attackontitan.fandom.com/ru/wiki/%D0%A...,154.0,50.0,NaN,NaN,NaN,NaN,NaN


In [42]:
import pandas as pd
import requests
import time
from bs4 import BeautifulSoup

df = pd.read_csv('attackontitan_characters_info.csv')

needed_skills = ["Навыки боя", "Инициативность", "Интеллект", "Командная работа"]
for skill in needed_skills:
    if skill not in df.columns:
        df[skill] = None

missing_skills_df = df[df[needed_skills].isnull().any(axis=1)]

def extract_skills(html):
    soup = BeautifulSoup(html, "html.parser")
    skills = {skill: None for skill in needed_skills}

    for row in soup.find_all("tr"):
        skill_name_tag = row.find("th")
        skill_value_tags = row.find_all("th")

        if skill_name_tag and len(skill_value_tags) > 1:
            skill_name = skill_name_tag.text.strip()
            skill_value = skill_value_tags[-1].text.strip()

            if skill_name in needed_skills:
                skills[skill_name] = skill_value

    return skills


for index, row in missing_skills_df.iterrows():
    char_name = row['name']
    char_url = row['link']

    if pd.isna(char_url) or not char_url.startswith("http"):
        print(f"Пропущено: {char_name} (отсутствует корректный URL)")
        continue

    try:
        response = requests.get(char_url)
        response.raise_for_status()
        soup = BeautifulSoup(response.text, 'html.parser')

        skills = extract_skills(str(soup))

        for skill, value in skills.items():
            df.at[index, skill] = value

        print(f"Обновлено: {char_name}")

    except requests.RequestException as e:
        print(f"Ошибка запроса для {char_name}: {e}")

    time.sleep(1)

# Сохраняем обновленный датасет
df.to_csv('skills_info_updated.csv', index=False, encoding='utf-8')


Обновлено: Абель
Обновлено: Абель Рейсс
Обновлено: Альма
Обновлено: Анка Рейнбергер
Обновлено: Армин Арлерт
Обновлено: Артур Браус
Обновлено: Аурилл
Обновлено: Безымянный медик
Обновлено: Бертольд Гувер
Обновлено: Валдо Рихтер
Обновлено: Вальц
Обновлено: Вилли Тайбер
Обновлено: Вэйн Эйснер
Обновлено: Г-жа Аккерман
Обновлено: Г-жа Арлерт
Обновлено: Г-жа Браун
Обновлено: Г-жа Кирштейн
Обновлено: Г-жа Спрингер
Обновлено: Г-н Аккерман
Обновлено: Г-н Арлерт
Обновлено: Г-н Рал
Обновлено: Г-н Рейсс
Обновлено: Г-н Смит
Обновлено: Г-н Спрингер
Обновлено: Габи Браун
Обновлено: Гергер
Обновлено: Герос
Обновлено: Гордон
Обновлено: Гриша Йегер
Обновлено: Гросс
Обновлено: Густав
Обновлено: Гюнтер Шульц
Обновлено: Даз
Обновлено: Дариус Байер-Варбрун
Обновлено: Дариус Закклай
Обновлено: Девушка из Культа стен
Обновлено: Дед Аккерман
Обновлено: Дед Арлерт
Обновлено: Делтофф
Обновлено: Демонический титан
Обновлено: Деннис Айбрингер
Обновлено: Джер Санес
Обновлено: Димо Ривз
Обновлено: Дина Фриц
Обновлен

In [43]:
df

,name,link,Рост,Вес,Пол,Статус,Убито титанов Соло,Убито титанов В команде,Убито титанов Всего,Навыки боя,Инициативность,Интеллект,Командная работа
0,Абель,https://attackontitan.fandom.com/ru/wiki/%D0%9...,175.0,62.0,NaN,NaN,NaN,NaN,NaN,None,None,None,None
1,Абель Рейсс,https://attackontitan.fandom.com/ru/wiki/%D0%9...,NaN,NaN,Женский,Мертв,NaN,NaN,NaN,None,None,None,None
2,Альма,https://attackontitan.fandom.com/ru/wiki/%D0%9...,168.0,54.0,Женский,Мертв,NaN,NaN,NaN,None,None,None,None
3,Анка Рейнбергер,https://attackontitan.fandom.com/ru/wiki/%D0%9...,155.0,53.0,NaN,NaN,NaN,NaN,NaN,None,None,None,None
4,Армин Арлерт,https://attackontitan.fandom.com/ru/wiki/%D0%9...,169.0,56.0,NaN,NaN,2.0,NaN,2.0,2/10,6/10,10/10,8/10
...,...,...,...,...,...,...,...,...,...,...,...,...,...
139,Фрида Рейсс,https://attackontitan.fandom.com/ru/wiki/%D0%A...,171.0,57.0,NaN,NaN,NaN,NaN,NaN,None,None,None,None
140,Фриц (Подставной),https://attackontitan.fandom.com/ru/wiki/%D0%A...,177.0,62.0,NaN,NaN,NaN,NaN,NaN,None,None,None,None
141,Ханджи Зое,https://attackontitan.fandom.com/ru/wiki/%D0%A...,170.0,60.0,NaN,NaN,5.0,NaN,5.0,7/10,10/10,10/10,8/10
142,Ханна Диамант,https://attackontitan.fandom.com/ru/wiki/%D0%A...,154.0,50.0,NaN,NaN,NaN,NaN,NaN,None,None,None,None


In [46]:
titan_count = (df["Убито титанов Всего"].isnull()).sum()
print(f"Количество NULL в 'Кол-во убитых титанов': {titan_count}")

titan_count = (df["Навыки боя"].isnull()).sum()
print(f"Количество NULL в 'Навыки боя': {titan_count}")

titan_count = (df["Инициативность"].isnull()).sum()
print(f"Количество NULL в 'Инициативность': {titan_count}")

titan_count = (df["Интеллект"].isnull()).sum()
print(f"Количество NULL в 'Интеллект': {titan_count}")

titan_count = (df["Командная работа"].isnull()).sum()
print(f"Количество NULL в 'Командная работа': {titan_count}")

Количество NULL в 'Кол-во убитых титанов': 120
Количество NULL в 'Навыки боя': 119
Количество NULL в 'Инициативность': 114
Количество NULL в 'Интеллект': 118
Количество NULL в 'Командная работа': 119


Показатели почти сходятся, это хорошо.

Немного подкорректируем датасет

In [50]:
import pandas as pd

def normalize_status(status):
    if pd.isna(status):
        return status  # Оставляем NaN без изменений
    status = status.lower().strip()

    if status in ["мёртв", "покойный", "мёртвманганеизвестно","мертв"]:
        return "Мертв"
    elif status in ["жив", "жива"]:
        return "Жив"
    elif status in ["неизвестно"]:
        return "Неизвестно"
    return status

df["Статус"] = df["Статус"].apply(normalize_status)
df.to_csv('status_info.csv', index=False, encoding='utf-8')
df

,name,link,Рост,Вес,Пол,Статус,Убито титанов Соло,Убито титанов В команде,Убито титанов Всего,Навыки боя,Инициативность,Интеллект,Командная работа
0,Абель,https://attackontitan.fandom.com/ru/wiki/%D0%9...,175.0,62.0,NaN,NaN,NaN,NaN,NaN,None,None,None,None
1,Абель Рейсс,https://attackontitan.fandom.com/ru/wiki/%D0%9...,NaN,NaN,Женский,Мертв,NaN,NaN,NaN,None,None,None,None
2,Альма,https://attackontitan.fandom.com/ru/wiki/%D0%9...,168.0,54.0,Женский,Мертв,NaN,NaN,NaN,None,None,None,None
3,Анка Рейнбергер,https://attackontitan.fandom.com/ru/wiki/%D0%9...,155.0,53.0,NaN,NaN,NaN,NaN,NaN,None,None,None,None
4,Армин Арлерт,https://attackontitan.fandom.com/ru/wiki/%D0%9...,169.0,56.0,NaN,NaN,2.0,NaN,2.0,2/10,6/10,10/10,8/10
...,...,...,...,...,...,...,...,...,...,...,...,...,...
139,Фрида Рейсс,https://attackontitan.fandom.com/ru/wiki/%D0%A...,171.0,57.0,NaN,NaN,NaN,NaN,NaN,None,None,None,None
140,Фриц (Подставной),https://attackontitan.fandom.com/ru/wiki/%D0%A...,177.0,62.0,NaN,NaN,NaN,NaN,NaN,None,None,None,None
141,Ханджи Зое,https://attackontitan.fandom.com/ru/wiki/%D0%A...,170.0,60.0,NaN,NaN,5.0,NaN,5.0,7/10,10/10,10/10,8/10
142,Ханна Диамант,https://attackontitan.fandom.com/ru/wiki/%D0%A...,154.0,50.0,NaN,NaN,NaN,NaN,NaN,None,None,None,None


In [52]:
df = df[~df['name'].isin(['Лошади', 'Родители Габи'])]

In [53]:
df

,name,link,Рост,Вес,Пол,Статус,Убито титанов Соло,Убито титанов В команде,Убито титанов Всего,Навыки боя,Инициативность,Интеллект,Командная работа
0,Абель,https://attackontitan.fandom.com/ru/wiki/%D0%9...,175.0,62.0,NaN,NaN,NaN,NaN,NaN,None,None,None,None
1,Абель Рейсс,https://attackontitan.fandom.com/ru/wiki/%D0%9...,NaN,NaN,Женский,Мертв,NaN,NaN,NaN,None,None,None,None
2,Альма,https://attackontitan.fandom.com/ru/wiki/%D0%9...,168.0,54.0,Женский,Мертв,NaN,NaN,NaN,None,None,None,None
3,Анка Рейнбергер,https://attackontitan.fandom.com/ru/wiki/%D0%9...,155.0,53.0,NaN,NaN,NaN,NaN,NaN,None,None,None,None
4,Армин Арлерт,https://attackontitan.fandom.com/ru/wiki/%D0%9...,169.0,56.0,NaN,NaN,2.0,NaN,2.0,2/10,6/10,10/10,8/10
...,...,...,...,...,...,...,...,...,...,...,...,...,...
139,Фрида Рейсс,https://attackontitan.fandom.com/ru/wiki/%D0%A...,171.0,57.0,NaN,NaN,NaN,NaN,NaN,None,None,None,None
140,Фриц (Подставной),https://attackontitan.fandom.com/ru/wiki/%D0%A...,177.0,62.0,NaN,NaN,NaN,NaN,NaN,None,None,None,None
141,Ханджи Зое,https://attackontitan.fandom.com/ru/wiki/%D0%A...,170.0,60.0,NaN,NaN,5.0,NaN,5.0,7/10,10/10,10/10,8/10
142,Ханна Диамант,https://attackontitan.fandom.com/ru/wiki/%D0%A...,154.0,50.0,NaN,NaN,NaN,NaN,NaN,None,None,None,None


In [54]:
df.to_csv('attackontitan_info_characters.csv', index=False, encoding='utf-8')